In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

rng = np.random.default_rng(19454)

pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_dark+presentation"

# Contingency

In [ ]:
df = pd.DataFrame(
    {
        "x": ["c", "a", "b", "a", "b", "b"],
        "y": [
            "R",
            "S",
            "S",
            "T",
            "R",
            "S",
        ],
    }
)
print(df.to_markdown())

In [ ]:
print(df["x"].value_counts().sort_index().to_markdown())

In [ ]:
print(df["y"].value_counts().sort_index().to_markdown())

In [ ]:
print(pd.crosstab(df["x"], df["y"]).to_markdown())

In [ ]:
fig_discrete = df.plot.scatter(x=df["x"], y=df["y"])
fig_discrete.write_image("strategies/screencast/public/fig_discrete.svg")
fig_discrete

In [ ]:
fig_heatmap_small = px.imshow(pd.crosstab(df["x"], df["y"]))
fig_heatmap_small.write_image("strategies/screencast/public/fig_heatmap_small.svg")
fig_heatmap_small.show()

In [ ]:
# Initialize the DataFrame
columns = ["poor", "fair", "good", "very good", "excellent"]
index = range(11)

# Create a base array with desired peak and decay
data = np.zeros((11, 5), dtype=int)

# Set the peak value
data[8, 3] = 5000  # "very good", index 8

# Apply decay in both dimensions
for i in range(11):
    for j in range(5):
        # Calculate decay based on distance from the peak (8 for index, 3 for column)
        distance_from_peak = abs(i - 8) + abs(j - 3)
        data[i, j] = max(0, 5000 - (distance_from_peak * 500))

# Fill in the rest of the DataFrame with random counts for spots that are not zero
for i in range(11):
    for j in range(5):
        data[i, j] = max(rng.integers(1, 100), data[i, j] + rng.integers(0, 1000))

# Create DataFrame
df = pd.DataFrame(data, columns=columns, index=index)


print(df.to_markdown())

In [ ]:
fig_heatmap_large = px.imshow(df)
fig_heatmap_large.update_yaxes(autorange=True)
fig_heatmap_large.write_image("strategies/screencast/public/fig_heatmap_large.svg")
fig_heatmap_large.show()

In [ ]:
fig_heatmap_large_w_labels = px.imshow(df, text_auto=True)
fig_heatmap_large_w_labels.update_yaxes(autorange=True)
fig_heatmap_large_w_labels.write_image(
    "strategies/screencast/public/fig_heatmap_large_w_labels.svg"
)
fig_heatmap_large_w_labels.show()

# Continuous data

In [ ]:
def get_c(n_draws):
    return pd.DataFrame(
        rng.multivariate_normal([0, 0], [[1, 0.3], [0.3, 1]], n_draws),
        columns=["x", "y"],
    )


dfs = {n: get_c(n) for n in [100, 10_000]}

In [ ]:
print(dfs[100].head(3).to_markdown())
print(*(["|"] * 4), sep=" ... ")
print("\n".join(dfs[100].tail(3).to_markdown().split("\n")[2:]))

In [ ]:
figs = {n: d.plot.scatter(x="x", y="y") for n, d in dfs.items()}

In [ ]:
[
    fig.update_layout(
        xaxis_range=[-5, 5],
        yaxis_range=[-5, 5],
    ).write_image(f"strategies/screencast/public/fig_continuous_{n}.svg")
    for n, fig in figs.items()
]

# Discrete + continuos 

In [ ]:
def get_dc(n):
    n = [1_000]
    return pd.DataFrame(
        rng.multivariate_normal(
            [0, 0.3, -0.2], [[1, 0.3, -0.2], [0.3, 1, 0.7], [-0.2, 0.7, 1]], n
        ),
        columns=["a", "b", "c"],
    ).melt(var_name="x", value_name="y")

In [ ]:
df = get_dc(1_000)

fig_dc = df.plot.scatter(x="x", y="y")
fig_dc.write_image("strategies/screencast/public/fig_dc.svg")
fig_dc

In [ ]:
fig_box = df.plot.box(x="x", y="y")
fig_box.write_image("strategies/screencast/public/fig_box.svg")
fig_box

In [ ]:
fig_violin = px.violin(df, x="x", y="y")
fig_violin.write_image("strategies/screencast/public/fig_violin.svg")
fig_violin

# Covariance

In [ ]:
tiny = pd.DataFrame({"A": [2, 4, 6], "B": [1, 3, 8]})

In [ ]:
print(tiny.to_markdown(index=False))

In [ ]:
fig_orig = tiny.plot.scatter(x="A", y="B")
fig_orig.update_layout(
    xaxis_range=[0, 10],
    yaxis_range=[0, 10],
    showlegend=False,
)
fig_orig.write_image("covariance/screencast/public/fig_orig.svg")
fig_orig

In [ ]:
tiny["A - 4"] = tiny["A"] - 4
tiny["B - 4"] = tiny["B"] - 4
tiny["(A - 4)(B - 4)"] = tiny["A - 4"] * tiny["B - 4"]
print(tiny.to_markdown(index=False))

In [ ]:
tiny_exchanged = tiny[["A", "B"]].copy()
tiny_exchanged["B"] = [8, 3, 1]
print(tiny_exchanged.to_markdown(index=False))

In [ ]:
fig_exchanged = tiny_exchanged.plot.scatter(x="A", y="B")
fig_exchanged.update_layout(
    xaxis_range=[0, 10],
    yaxis_range=[0, 10],
    showlegend=False,
)
fig_exchanged.write_image("covariance/screencast/public/fig_exchanged.svg")
fig_exchanged

In [ ]:
tiny_exchanged["A - 4"] = tiny_exchanged["A"] - 4
tiny_exchanged["B - 4"] = tiny_exchanged["B"] - 4
tiny_exchanged["(A - 4)(B - 4)"] = tiny_exchanged["A - 4"] * tiny_exchanged["B - 4"]
print(tiny_exchanged.to_markdown(index=False))

# Correlations

In [ ]:
def get_corr_c(corr, n):
    means = np.array([1.05, -10])
    chol = np.linalg.cholesky(np.array([[0.1**2, corr / 100], [corr / 100, 10**2]]))
    return pd.DataFrame(
        means + (chol @ rng.multivariate_normal([0, 0], [[1, 0], [0, 1]], n).T).T,
        columns=["x", "y"],
    )


dfs = {c: get_corr_c(c, 100) for c in [90, 30, 0, -70, -99]}

In [ ]:
for c, df in dfs.items():
    fig = df.plot.scatter(x="x", y="y")
    fig.update_layout(
        xaxis_range=[0.71, 1.39],
        yaxis_range=[-44, 24],
        showlegend=False,
        title=f"Correlation: {c/100:.2g}",
    )
    fig.write_image(f"correlation/screencast/public/fig_{c}.svg")
    fig.show()
    if c == 30:
        fig.add_trace(
            go.Scatter(
                x=[1.05],
                y=[-10],
                mode="markers",
                marker={"size": 20, "color": "red"},
            )
        )
        fig.write_image(f"correlation/screencast/public/fig_dot_{c}.svg")
        fig.show()

In [ ]:
dfs[c].describe()

In [ ]:
c = 30
df = dfs[c].copy()
df -= np.array([1.05, -10])
df

In [ ]:
fig_demeaned = df.plot.scatter(x="x", y="y")
fig_demeaned.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        mode="markers",
        marker={"size": 20, "color": "red"},
    )
)
fig_demeaned.update_layout(
    xaxis_range=[-0.34, 0.34],
    yaxis_range=[-34, 34],
    showlegend=False,
    title=f"Correlation: {c/100:.1f}, demeaned x and y",
)
fig_demeaned.write_image("correlation/screencast/public/fig_demeaned.svg")
fig_demeaned.show()

In [ ]:
df /= pd.Series({"x": 0.1, "y": 10})
fig_standardised = df.plot.scatter(x="x", y="y")
fig_standardised.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        mode="markers",
        marker={"size": 20, "color": "red"},
    )
)
fig_standardised.update_layout(
    xaxis_range=[-3.4, 3.4],
    yaxis_range=[-3.4, 3.4],
    showlegend=False,
    title=f"Correlation: {c/100:.1f}, demeaned x and y with unit variance",
)
fig_standardised.write_image("correlation/screencast/public/fig_standardised.svg")
fig_standardised.show()

# OLS

In [ ]:
df = get_dc(10_000)
fig_violin = px.violin(df, x="x", y="y")

fig_violin.add_trace(
    go.Scatter(
        x=["a", "b", "c"],
        y=df.groupby("x")["y"].mean(),
        mode="markers",
        marker={"size": 10, "color": "red"},
    )
)
fig_violin.update_layout(
    showlegend=False,
    title="Means of y given discrete x",
)

fig_violin.write_image("ols_intuition/screencast/public/fig_violin.svg")
fig_violin

In [ ]:
df = get_corr_c(30, 300)

In [ ]:
fig = df.plot.scatter(x="x", y="y")
fig.add_trace(
    go.Scatter(
        x=[df["x"].mean()],
        y=[df["y"].mean()],
        mode="markers",
        marker={"size": 20, "color": "red"},
    )
)
fig.update_layout(
    showlegend=False,
    title="Overall mean of y and x",
)
fig.write_image("ols_intuition/screencast/public/fig_cont_overall_mean.svg")
fig.show()

In [ ]:
for i in range(2, 7):
    qcut, bins = pd.qcut(df["x"], i, retbins=True)
    fig = df.plot.scatter(x="x", y="y")
    fig.add_trace(
        go.Scatter(
            x=df.groupby(qcut)["x"].mean(),
            y=df.groupby(qcut)["y"].mean(),
            mode="markers",
            marker={"size": 20, "color": "red"},
        )
    )
    for b in bins[1:-1]:
        fig.add_vline(x=b, line_width=2, line_color="red")
    fig.update_layout(
        showlegend=False,
        title=f"Means of y in {i} bins of x",
    )
    fig.write_image(f"ols_intuition/screencast/public/fig_cont_means_{i}.svg")
    fig.show()

In [ ]:
def get_reg_line(df, dev):
    slope = df.cov().loc["x", "y"] / df["x"].var() + dev
    intercept = df["y"].mean() - slope * df["x"].mean()
    x_points = [df["x"].min(), 0.97, 1.2, df["x"].max()]
    return {
        "x": x_points,
        "y": [slope * x + intercept for x in x_points],
        "slope": slope,
    }

In [ ]:
for slope_change in range(-35, 36, 5):
    fig = df.plot.scatter(x="x", y="y")
    fig.add_trace(
        go.Scatter(
            x=[df["x"].mean()],
            y=[df["y"].mean()],
            mode="markers",
            marker={"size": 20, "color": "red"},
        )
    )

    line = get_reg_line(df=df, dev=slope_change)
    # Add the line to the figure
    fig.add_trace(
        go.Scatter(x=line["x"], y=line["y"], mode="lines", line={"color": "red"})
    )

    fig.update_layout(
        showlegend=False,
        title=f"Line through means of x and y, slope {line['slope']:.1f}",
    )
    fig.write_image(f"ols_intuition/screencast/public/fig_cont_line_{slope_change}.svg")
    fig.show()

In [ ]:
df_sel = df.copy()
df_sel.iloc[0] = {"x": 1.2, "y": 12}
df_sel.iloc[1] = {"x": 0.97, "y": -35}